# Blinkit SKU Tracker

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ExpectedConditions
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from time import sleep
import time
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

In [ ]:
df=pd.read_excel('Blinkit Data.xlsx')
df

In [ ]:
mobile_emulation= {"deviceName":"Nexus 5"}
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
options = Options()

options.add_experimental_option("mobileEmulation",mobile_emulation)
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')

In [ ]:
new={"Pincode":[], "City":[],"Master SKU":[],"PRID":[], "Title":[],"SP":[],"Opportunity Lost":[]}
count = 0
start_time = time.time()

In [ ]:
for i in range(0,10):
  
    driver=webdriver.Chrome(executable_path=path,options=options)
    

    driver.get(df.loc[i,"Link"])
    sleep(3)
    
    try:
        driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]').click()  
        sleep(1)
        
    except NoSuchElementException:      
        pass
    
    try:
        false = driver.find_element_by_xpath('//*[@id="app"]/div/div/div[3]/div/div/section/h1')
        new["Pincode"].append(df["Pincode"][i])
        new["City"].append(df["city"][i])
        new["Master SKU"].append(df["Master SKU"][i])
        new["PRID"].append(df["product_id"][i])
        new["Opportunity Lost"].append(df["Opportunity Lost"][i])
        new["Title"].append(df["Master SKU"][i])
        new["SP"].append("Out of Stock")
    
    
    except NoSuchElementException:
        driver.find_element_by_xpath('//*[@id="app"]/div/div/div[1]/header/div[1]/div[2]/div/div[2]/div[1]').click()
        pincode = driver.find_element_by_xpath('//*[@id="react-select-2--value"]/div[2]/input')
        sleep(1)
        pin = str(df.loc[i,"Pincode"])
        pincode.send_keys(pin)
        sleep(1)
        pincode.send_keys(" ")
        sleep(3)
        pincode.send_keys(Keys.RETURN)
        sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        title = soup.find("h1", class_="ProductInfoCard__ProductName-sc-113r60q-10 eUWnwG")
        container = soup.find("div", class_="ProductVariants__VariantsCardSlider-sc-1unev4j-1 fpFisu")

        if title:
            new["Title"].append(title.text)
        else:
            new["Title"].append("N/A")

        if container:
            in_stock = False
            for box in container:
                price = box.find("div", class_="ProductVariants__PriceContainer-sc-1unev4j-6 gldKTp").text
                if "₹" in price:
                    price = price.split()[0]
                    new["SP"].append(price)
                    in_stock = True
                    break
            if not in_stock:
                new["SP"].append("Out of Stock")
        else:
            new["SP"].append("N/A")
            
        new["Pincode"].append(df["Pincode"][i])
        new["City"].append(df["city"][i])
        new["Master SKU"].append(df["Master SKU"][i])
        new["PRID"].append(df["product_id"][i])
        new["Opportunity Lost"].append(df["Opportunity Lost"][i])        
    
    count = count + 1
    print(count)
    
    driver.close()

    
end_time = time.time()
print("Start Time: ",time.ctime(int(start_time)))
print("End Time: ",time.ctime(int(end_time)))
print(f"Total runtime of the program is {end_time - start_time}")


In [ ]:
res = pd.DataFrame(new)